# DX 704 Week 2 Project

This week's project will analyze fresh strawberry price data for a hypothetical "buy low, freeze, and sell high" business.
Strawberries show strong seasonality in their prices compared to other fruits.

![](https://ers.usda.gov/sites/default/files/_laserfiche/Charts/61401/oct14_finding_plattner_fig01.png)

Image source: https://www.ers.usda.gov/amber-waves/2014/october/seasonal-fresh-fruit-price-patterns-differ-across-commodities-the-case-of-strawberries-and-apples

You are considering a business where you buy strawberries when the prices are very low, carefully freeze them, even more carefully defrost them, and then sell them when the prices are high.
You will forecast strawberry price time series and then use them to tactically pick times to buy, freeze, and sell the strawberries.

The full project description, a template notebook, and raw data are available on GitHub at the following link.

https://github.com/bu-cds-dx704/dx704-project-02


### Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Backtest Strawberry Prices

Read the provided "strawberry-prices.tsv" with data from 2020 through 2024.
This data is based on data from the U.S. Bureau of Statistics, but transformed so the ground truth is not online.
https://fred.stlouisfed.org/series/APU0000711415

Use the data for 2020 through 2023 to predict monthly prices in 2024.
Spend some time to make sure you are happy with your methodology and prediction accuracy, since you will reuse the methodology to forecast 2025 next.
Save the 2024 backtest predictions as "strawberry-backtest.tsv" with columns month and price.

In [2]:
# imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [3]:
# YOUR CHANGES HERE

strawb = pd.read_csv('strawberry-prices.tsv',sep="\t")
strawb

,month,price
0,2020-01-01,4.0490
1,2020-02-01,3.6250
2,2020-03-01,3.3770
3,2020-04-01,3.2328
4,2020-05-01,3.1260
5,2020-06-01,2.9260
6,2020-07-01,2.9260
7,2020-08-01,3.1780
8,2020-09-01,3.2040
9,2020-10-01,3.8690


In [4]:
!pip install scikit-learn

In [5]:
from sklearn.linear_model import LinearRegression

# copy dataset
df = strawb.copy()

# convert month to datetime (just for modeling not for saved file)
df['month'] = pd.to_datetime(df['month'])

# create time index
df['t'] = np.arange(len(df))

# add seasonality features
df['sin12'] = np.sin(2 * np.pi * df['t'] / 12)
df['cos12'] = np.cos(2 * np.pi * df['t'] / 12)

# train on 2020–2023
train = df[df['month'] < '2024-01-01']
test = df[df['month'] >= '2024-01-01']

X_train = train[['t', 'sin12', 'cos12']]
y_train = train['price']

X_test = test[['t', 'sin12', 'cos12']]

# train model
model = LinearRegression()
model.fit(X_train, y_train)

# predict 2024
pred_2024 = model.predict(X_test)

# save backtest results
backtest = pd.DataFrame({
    'month': test['month'].dt.strftime('%Y-%m-01'),
    'price': pred_2024
})

backtest.to_csv("strawberry-backtest.tsv", sep="\t", index=False)

backtest

,month,price
48,2024-01-01,4.804419
49,2024-02-01,4.662483
50,2024-03-01,4.402729
51,2024-04-01,4.098754
52,2024-05-01,3.836005
53,2024-06-01,3.688882
54,2024-07-01,3.700802
55,2024-08-01,3.872569
56,2024-09-01,4.162154
57,2024-10-01,4.495960


Please use the same format for the month column as in the training data, i.e. YYYY-MM-01.
The autograder may not be able to parse other formats.

Submit "strawberry-backtest.tsv" in Gradescope.

## Part 2: Backtest Errors

What are the mean and standard deviation of the residuals between your backtest predictions and the ground truth?

Write the mean and standard deviation to a file "backtest-accuracy.tsv" with two columns, mean and std.

In [6]:
residuals = test['price'].values - backtest['price'].values

mean_resid = residuals.mean()
std_resid = pd.Series(residuals).std()   # <-- FIX

accuracy = pd.DataFrame({
    'mean': [mean_resid],
    'std': [std_resid]
})

accuracy.to_csv("backtest-accuracy.tsv", sep="\t", index=False)

accuracy


,mean,std
0,-0.452316,0.284362


Hint: If the mean residual in your backtest is not close to zero, then your model is likely missing a systematic change and you should go back to improve it.

Submit "backtest-accuracy.tsv" in Gradescope.

## Part 3: Forecast Strawberry Prices

Use all the data from 2020 through 2024 to predict monthly prices in 2025 using the same methodology from part 1.
Make a monthly forecast for each month of 2025 and save it as "strawberry-forecast.tsv" with columns for month and price.


In [7]:
# YOUR CHANGES HERE

# retrain w ALL data 2020–2024
X_all = df[['t', 'sin12', 'cos12']]
y_all = df['price']

model_final = LinearRegression()
model_final.fit(X_all, y_all)

# create future months 2025
last_t = df['t'].iloc[-1]

future_t = np.arange(last_t + 1, last_t + 13)

future_months = pd.date_range(start='2025-01-01', periods=12, freq='MS')

future_df = pd.DataFrame({
    't': future_t
})

# seasonal features
future_df['sin12'] = np.sin(2 * np.pi * future_df['t'] / 12)
future_df['cos12'] = np.cos(2 * np.pi * future_df['t'] / 12)

# predict 2025
pred_2025 = model_final.predict(future_df)

# save
forecast = pd.DataFrame({
    'month': future_months.strftime('%Y-%m-01'),
    'price': pred_2025
})

forecast.to_csv("strawberry-forecast.tsv", sep="\t", index=False)

forecast

,month,price
0,2025-01-01,4.704594
1,2025-02-01,4.529045
2,2025-03-01,4.231618
3,2025-04-01,3.893988
4,2025-05-01,3.608602
5,2025-06-01,3.453909
6,2025-07-01,3.473339
7,2025-08-01,3.663664
8,2025-09-01,3.975867
9,2025-10-01,4.328273


Submit "strawberry-forecast.tsv" in Gradescope.

## Part 4: Buy Low, Freeze and Sell High

Using your 2025 forecast, analyze the profit picking different pairs of months to buy and sell strawberries.
Maximize your profit assuming that it costs &dollar;0.20 per pint to freeze the strawberries, &dollar;0.10 per pint per month to store the frozen strawberries and there is a 10% price discount from selling previously frozen strawberries.
So, if you buy a pint of strawberies for &dollar;1, freeze them, and sell them for &dollar;2 three months after buying them, then the profit is &dollar;2 * 0.9 - &dollar;1 - &dollar;0.20 - &dollar;0.10 * 3 = &dollar;0.30 per pint.
To evaluate a given pair of months, assume that you can invest &dollar;1,000,000 to cover all costs, and that you buy as many pints of strawberries as possible.

Write the results of your analysis to a file "timings.tsv" with columns for the buy_month, sell_month, pints_purchased, and expected_profit.

In [8]:
# YOUR CHANGES HERE

budget = 1_000_000

results = []

# buy - sell month pairs
for i in range(len(forecast)):
    for j in range(i + 1, len(forecast)):
        
        buy_month = forecast.iloc[i]['month']
        sell_month = forecast.iloc[j]['month']
        
        buy_price = forecast.iloc[i]['price']
        sell_price = forecast.iloc[j]['price']
        
        months_stored = j - i
        
        # profit per pint
        profit_per_pint = (
            0.9 * sell_price 
            - buy_price 
            - 0.20 
            - 0.10 * months_stored
        )
        
        # cost per pint
        cost_per_pint = buy_price + 0.20 + 0.10 * months_stored
        
        # max pints purchasable
        pints = int(budget // cost_per_pint)
        
        expected_profit = pints * profit_per_pint
        
        results.append([
            buy_month,
            sell_month,
            pints,
            expected_profit
        ])

# convert to df
results_df = pd.DataFrame(
    results,
    columns=['buy_month', 'sell_month', 'pints_purchased', 'expected_profit']
)

# find best strategy
results_df = results_df.sort_values(by='expected_profit', ascending=False)

# save best result (top row)
best_trade = results_df.head(1)
best_trade.to_csv("timings.tsv", sep="\t", index=False)

best_trade


,buy_month,sell_month,pints_purchased,expected_profit
55,2025-07-01,2025-12-01,239616,34690.232802


Submit "timings.tsv" in Gradescope.

## Part 5: Strategy Check

What is the best profit scenario according to your previous timing analysis?
How much does that profit change if the sell price is off by one standard deviation from your backtest analysis?
(Variation in the sell price is more dangerous because you can see the buy price before fully committing.)

Write the results to a file "check.tsv" with columns `best_profit` and `one_std_profit`.
To be clear, `one_std_profit` should be the number of pints bought in your best profit scenario times your backtested standard deviation of the residual.
This represents the standard deviation in revenue when selling if you explicitly assume that you buy according to the best profit scenario and your backtest standard deviation is representative of the future prices.

In [9]:
# YOUR CHANGES HERE

# best profit scenario
best_profit = best_trade['expected_profit'].iloc[0]
pints = best_trade['pints_purchased'].iloc[0]

# add uncertainty
one_std_profit = pints * std_resid

# save
check = pd.DataFrame({
    'best_profit': [best_profit],
    'one_std_profit': [one_std_profit]
})

check.to_csv("check.tsv", sep="\t", index=False)

check

,best_profit,one_std_profit
0,34690.232802,68137.756426


Submit "check.tsv" in Gradescope.

*sooooo if the sell price is off by ±1 standard deviation, total revenue changes by about $68k. uncertainty is almost double expected profit!!! this is bad*

## Part 6: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgments are appropriate, just write none in the file.


Submit "acknowledgments.txt" in Gradescope.

In [10]:
content = (
    "Acknowledgments\n\n"
    "I used the DX704 course-provided code examples as a reference when developing "
    "the code for this week's assignment.\n\n"
    "Specifically, I referenced the following notebook from the course repository:\n"
    "https://github.com/bu-cds-omds/dx704-examples/blob/main/week01/"
    "chart_risk_return_performance_of_efficient_portfolios.ipynb\n"
)

with open("acknowledgments.txt", "w") as f:
    f.write(content)

## Part 7: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

Submit "project.ipynb" in Gradescope.